# **Tensorflow Datasets and Keras Models**

The aim of this notebook is to do several tests with some Tensorflow datasets and Keras models, that are going to be detailed hereunder.
The prerequisites for these tests are installing Tensorflow datasets and Keras, as well as all the modules that are considered necessary for the different sections.

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

# Commonly used modules
import numpy as np
import os
import sys

# Images, plots, display, and visualization
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import cv2
import imutils
import IPython
from six.moves import urllib

!pip install scikit-image
import skimage
from skimage import metrics

#Download tensorflow datasets
!pip install tensorflow-datasets
import tensorflow_datasets as tfds

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Switch function implementation
Hereunder, the implementation of a typical switch function is proposed. It is going to be useful later on to choose among the available Keras models to be generated.

In [ ]:
class switch:

	def __init__(self, variable, comparator=None, strict=False):
		self.variable = variable
		self.matched = False
		self.matching = False
		if comparator:
			self.comparator = comparator
		else:
			self.comparator = lambda x, y: x == y
		self.strict = strict

	def __enter__(self):
		return self

	def __exit__(self, exc_type, exc_val, exc_tb):
		pass

	def case(self, expr, break_=False):
		if self.strict:
			if self.matched:
				return False
		if self.matching or self.comparator(self.variable, expr):
			if not break_:
				self.matching = True
			else:
				self.matched = True
				self.matching = False
			return True
		else:
			return False

	def default(self):
		return not self.matched and not self.matching

## Preprocessing block
In this code block, a set of functions to preprocess the data of the different datasets are implemented.
For that, a batch size of 32 examples is defined.

The *configure_for_performance* function, prepares the data of the chosen dataset to be trained.

For that, first it calls the function *input_preprocess* to appply one-hot encoding to the train and test labels.
Then, it caches the dataset in order to save some operations, such as data reading or file opening, from being executed during each epoch.

It also shuffles the data, allocating a buffer size of 1000 to pick random entries and by means of the *Dataset.batch* function, stacks 32 consecutive elements of the dataset into a single element.

Finally, it applies prefetching to overlap the preprocessing and model execution of training steps. In this way, while the model is executing training step n, the input pipeline will be reading the data for step n+1, thus reducing the step time of the training and the time to extract the data.

The *resize_images* function uses lambda transformations to resize the images of the used dataset so that they have a shape that matches the input shape of the chosen model.

In [ ]:
#Preprocessing functions block
from tensorflow.python.data.ops.dataset_ops import AUTOTUNE
os.system("cls")

batch_size = 32

def configure_for_performance(ds):
  ds=ds.map(input_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
  ds=ds.cache()  #cache dataset
  ds=ds.shuffle(buffer_size=1000)
  ds=ds.batch(batch_size=batch_size)
  ds=ds.prefetch(buffer_size=AUTOTUNE)
  return (ds)


def input_preprocess(image, label):
    label = tf.one_hot(tf.cast(label,tf.int64), NUM_CLASSES)
    return image, label


def resize_imgs(IMG_SIZE, ds_train, ds_test, ds_val):
  size = (IMG_SIZE, IMG_SIZE)
  #Resize images to input shape of used model
  ds_train = ds_train.map(lambda img, lbl:
                          (tf.image.resize(img, size, method='nearest'), lbl))
  ds_test = ds_test.map(lambda img, lbl:
                        (tf.image.resize(img, size, method='nearest'), lbl))
  if ds_val != None:
    ds_val= ds_val.map(lambda img, lbl:
                       (tf.image.resize(img, size), lbl))
    return ds_train, ds_test, ds_val
  else:
    return ds_train, ds_test

#Load available datasets
The following functions (*load_mnist*, *load_fashion_mnist*, *load_celeba* and *load_cifar10*) are used to load the chosen dataset in each case.
All of them have a similar structure, in which first the data is loaded from the TensorFlow datasets and splitted into train (validation) and test sets and then, the number of classes of each dataset is obtained by means of their corresponding metadata.
These functions return the train and test datasets, as well as the number of classes corresponding to the chosen dataset.

More information about the different datasets that have been used can be found in the following links:

*   MNIST: https://www.tensorflow.org/datasets/catalog/mnist
*   Fashion-MNIST: https://www.tensorflow.org/datasets/catalog/fashion_mnist
*   CelebA: https://www.tensorflow.org/datasets/catalog/celeb_a
*   CIFAR-10: https://www.tensorflow.org/datasets/catalog/cifar10

In [ ]:
#Functions for loading different TensorFlow datasets
def load_mnist():
  """This function loads MNIST dataset"""
  (ds_train, ds_test), ds_info = tfds.load(
      'mnist',
      split = ["train", "test"],
      with_info = True,
      as_supervised = True
      )
  num_classes = ds_info.features["label"].num_classes
  return(ds_train, ds_test, num_classes)


def load_fashion_mnist():
  """This function loads Fashion MNIST dataset"""
  (ds_train, ds_test), ds_info = tfds.load(
      'fashion_mnist',
      split = ['train', 'test'],
      with_info = True,
      as_supervised = True
      )
  num_classes = ds_info.features["label"].num_classes
  return(ds_train, ds_test, num_classes)


def load_celeba():
  """This function loads Celeb_a dataset"""
  (ds_train, ds_val, ds_test), ds_info = tfds.load(
      'celeb_a',
      split = ['train', 'validation', 'test'],
      with_info = True,
      as_supervised = True
      )
  num_classes = ds_info.features["label"].num_classes
  return(ds_train, ds_test, ds_val, num_classes)


def load_cifar10():
  """This function loads Cifar10 dataset"""
  (ds_train, ds_test), ds_info= tfds.load(
      'cifar10',
      split = ['train', 'test'],
      with_info = True,
      as_supervised = True
      )
  num_classes = ds_info.features["label"].num_classes
  return(ds_train, ds_test, num_classes)

#Generate available models
The function included hereunder (*get_model*) is used to get the chosen model among the imported Keras models.

The choice is performed by means of the previously included switch function, that selects the model to generate depending on the model name received at its input.

The function returns the generated model, which is already pre-trained.

More information about the different pre-trained models (https://keras.io/api/applications/) that have been imported can be found in the following links:
*   Resnet50: https://keras.io/api/applications/resnet/#resnet50-function
*   MobileNet: https://keras.io/api/applications/mobilenet/
*   DenseNet121: https://keras.io/api/applications/densenet/#densenet121-function
*   EfficientNetB0: https://keras.io/api/applications/efficientnet/#efficientnetb0-function

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess_input, decode_predictions

from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input as mobilenet_preprocess_input, decode_predictions

from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess_input, decode_predictions

from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess_input, decode_predictions

#Function to get the desired model
def get_model(model_name, classes):
  '''This function builds the desired model'''
  with switch(model_name) as m:
    if m.case('resnet', True): model=ResNet50(weights=None, classes=classes)
    if m.case('mobilenet', True): model=MobileNet(weights=None, classes=classes)
    if m.case('densenet', True): model=DenseNet121(weights=None, classes=classes)
    if m.case('efficientnet', True): model=EfficientNetB0(weights=None, classes=classes)
    if m.default(): print('error')
  return model

## Proposed pipeline
The aim of the following code block is to check the correct implementation of the functions explained above.

In it, first of all, the desired dataset is loaded using one of the functions prepared for this purpose.

Then, we take one example and check the image shape and its assigned label.

The next step is to resize images so that their shape match the input shape of the models. In this case, input shape of all models is 224 by 224.
To continue, the train and test datasets are preprocessed with the *configure_for_perfomance* function.

Once the datasets are preprocessed, the model is obtained using the *get_model* function, compiled and finally trained with the preprocessed train dataset to check the correct operation of the code developed.

In [ ]:
#Load the desired dataset
ds_train, ds_test, NUM_CLASSES = load_celeba()

In [ ]:
#Take an example
for image, label in ds_train.take(1):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())

Image shape:  (28, 28, 3)
Label:  29


In [ ]:
#Resize images to (224, 224)
IMG_SIZE = 224
ds_train, ds_test = resize_imgs(IMG_SIZE, ds_train, ds_test, ds_val=None)

In [ ]:
#Preprocess datasets
ds_train = configure_for_performance(ds_train)
ds_test = configure_for_performance(ds_test)

In [ ]:
#Compile desired model
model = get_model('resnet', NUM_CLASSES)
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam())

In [ ]:
#Train model
model.fit(ds_train) 

4/4 [==============================] - 21s 984ms/step - loss: 6.6013
